In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
cd  "drive/My Drive/Cmput 328/Assignment5"

[Errno 2] No such file or directory: 'drive/My Drive/Cmput 328/Assignment5'
/content/drive/My Drive/Cmput 328/Assignment5


In [10]:
!pwd

/content/drive/My Drive/Cmput 328/Assignment5


In [ ]:

import sys
#sys.path.append("drive/My Drive/Cmput 328/Assignment5")
import A5_utils

In [11]:
%run A5_main.py

Using pytorch version: 1.7.0+cu101
Using torchvision version: 0.8.1+cu101
Training on GPU: Tesla T4
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Training samples: 38000
Validation samples: 12000
Labeled training samples: 7600


  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  1%|▏         | 14/1000 [00:00<00:07, 136.63it/s]

Loading weights from: ./checkpoints/model.pt with:
	criterion: train_acc
	epoch: 44
	timestamp: 20/11/16 20:52:05

stats:
|       |   dscr_real |   cls_real |   cmp_real |   dscr_fake |   cls_fake |   cmp_fake |   cmp |   dscr_gen |   cls_gen |   cmp_gen |   total_acc |   real_acc |   fake_acc |   fid |    is |
|-------+-------------+------------+------------+-------------+------------+------------+-------+------------+-----------+-----------+-------------+------------+------------+-------+-------|
| train |       0.000 |      2.142 |      2.143 |       0.000 |      2.947 |      2.947 | 5.089 |     14.687 |     3.048 |    17.735 |      10.491 |     18.829 |      2.153 | 0.000 | 0.000 |
| valid |       0.000 |      0.000 |      0.000 |       0.000 |      0.000 |      0.000 | 0.000 |      0.000 |     0.000 |     0.000 |       0.000 |      0.000 |      0.000 | 0.000 | 0.000 |

Testing...
evaluating...


100%|██████████| 1000/1000 [00:07<00:00, 142.60it/s]


Test Results:

|      |   dscr_real |   cls_real |   cmp_real |   dscr_fake |   cls_fake |   cmp_fake |   cmp |   dscr_gen |   cls_gen |   cmp_gen |   total_acc |   real_acc |   fake_acc |    fid |     is |
|------+-------------+------------+------------+-------------+------------+------------+-------+------------+-----------+-----------+-------------+------------+------------+--------+--------|
| test |       0.000 |      1.431 |      1.431 |       0.000 |      1.474 |      1.474 | 2.905 |      7.301 |     1.524 |     8.825 |      15.595 |     21.190 |     10.000 | -1.000 | -1.000 |

Test Time: 2.610 sec


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/drive/My\ Drive/Cmput\ 328/Assignment5/tensorboard


In [ ]:
import torch
import pandas as pd
import torch.nn as nn

from A5_utils import REAL_UNLABELED, FAKE_UNLABELED, REAL_LABEL, FAKE_LABEL


class TrainParams:
    """
    :ivar n_workers: no. of threads for loading data

    :ivar validate_gap: gap in epochs between validations

    :ivar tb_path: folder where to save tensorboard data

    :ivar load_weights:
        0: train from scratch,
        1: load and test
        2: load if it exists and continue training

    """

    def __init__(self):
        self.n_workers = 0
        self.batch_size = 128
        self.n_epochs = 50
        self.load_weights = 0
        self.tb_path = './tensorboard'
        #self.weights_path ="drive/My Drive/Cmput 328/Assignment5"
        
        self.weights_path = './checkpoints/model.pt'
        self.validate_gap = 10


class OptimizerParams:
    def __init__(self):
        self.type = 0
        self.lr = 0.0002
        self.momentum = 0.9
        self.weight_decay = 0.0005
        self.beta1 = 0.5


class SharedNet(nn.Module):
    class Params:
        dummy = 0

    def __init__(self, params, n_channels=3, img_size=32):
        """

        :param SharedNet.Params params:
        """
        super(SharedNet, self).__init__()
        self.conv = nn.Sequential(
            # input size 3x32x32
            nn.Conv2d(n_channels,6,kernel_size=5),
            nn.LeakyReLU(0.2),
            # input size 6x28x28
            nn.Conv2d(6,12,kernel_size = 5),
            nn.BatchNorm2d(12),
            nn.LeakyReLU(0.2),
            # input size 12x20x20
            nn.Conv2d(12,24,kernel_size=5),
            nn.BatchNorm2d(24),
            nn.LeakyReLU(0.2)
            # output size 24x20x20
        )

        self.fc = nn.Sequential(
            nn.Linear(24*20*20,1024),
            nn.ReLU()
        )

        #pass

    def init_weights(self):
        def init_weight(m):
          cln = m.__class__.__name__
          if cln.find('Conv')!=-1:
            nn.init.normal_(m.weight.data,0.0,0.2)
          elif cln.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data,1.0,0.02)
            nn.init.constant_(m.bias.data,0)
        self.conv.apply(init_weight)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.shape[0],-1)
        x = self.fc(x)
        return x


class Discriminator(nn.Module):
    class Params:
        opt = OptimizerParams()

    def __init__(self, params, n_channels=3, img_size=32):
        """

        :param Discriminator.Params params:
        """
        super(Discriminator, self).__init__()
        #pass
        self.conv = nn.Sequential(
            # intput size 24x20x20
            nn.Linear(1024,1), 
            nn.Sigmoid()
        )

    def forward(self, x):
        """
        :param enc_input:
        :return:
        """
        #pass

        #x = x.view(x.shape[0],-1)
        output = self.conv(x)
        output = output.reshape(-1)
        return output

    def get_loss(self, dscr_out, labels):
        """

        :param dscr_out: Discriminator output
        :param labels: Real vs fake binary labels (real --> 1, fake --> 0)
        :return:
        """
        #pass
        #print('size of dscr_out is',len(dscr_out),'size of lables is',len(labels),'\n')
        #print(dscr_out,'\t','labels')
        
        loss = nn.BCELoss()
        return loss(dscr_out,labels)

    def get_optimizer(self, modules):
        """

        :param nn.ModuleList modules: [shared_net, discriminator, classifier, generator, composite_loss]
        :return:
        """
        opt = OptimizerParams()
        opt_params = (list(modules[0].parameters())+list(modules[1].parameters())+list(modules[2].parameters()))
        
        return torch.optim.Adam(opt_params, lr=opt.lr, betas=(opt.beta1,0.999), weight_decay=opt.weight_decay)

    def init_weights(self):
        pass


class Classifier(nn.Module):
    class Params:
        dummy = 0

    def __init__(self, params, n_classes=20, n_channels=3, img_size=32):
        """

        :param n_classes: 10 classes for real images and 10 for fake images
        :param Classifier.Params params:
        """
        super(Classifier, self).__init__()
        #pass
        self.net = nn.Sequential(
            nn.Linear(1024,20),
            #nn.Linear(256,20),
            nn.Softmax()
        )

    def get_loss(self, cls_out, labels, is_labeled=None, n_labeled=None):
        """

        :param cls_out: Classifier output
        :param labels: labels for both fake and real images in range 0 - 19;
        -1 for real unlabeled, -2 for fake unlabeled
        :param is_labeled: boolean array marking which labels are valid; None when all are valid
        :param n_labeled: number of valid labels;  None when all are valid
        :return:
        """
        #pass

        if n_labeled and n_labeled == 0:
          return torch.tensor(0.,requires_grad=True)

        loss_real = nn.CrossEntropyLoss(ignore_index=-1)
        loss_fake = nn.CrossEntropyLoss(ignore_index=-2)
        #temp = torch.tensor(0.,requires_grad=True)

        """for i in range(len(is_labeled)):
          if is_labeled[i] >= 0:
            print(cls_out[i].item,labels[i])
            temp += loss(cls_out[i],labels[i])"""

        if -1 in labels:
          #print('used -1 real')
          return loss_real(cls_out,labels)
        else:
          #print('used -2 fake')
          #print(loss_fake(cls_out,labels))
          return loss_fake(cls_out,labels)
        
        #return loss()
    def init_weights(self):
        pass

    def forward(self, x):
        #pass
        #x = x.view(x.shape[0],-1)
        x = self.net(x)
        #x = x.reshape(-1)
        return x

class Generator(nn.Module):
    class Params:
        opt = OptimizerParams()

    def __init__(self, params, input_size, n_channels=3, out_size=32):
        """

        :param Generator.Params params:
        """
        super(Generator, self).__init__()
        
        self.conv = nn.Sequential(
            nn.ConvTranspose2d(100,256,4,1,0,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( 128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( 64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, x):
      x = self.conv(x)
      #print("\n output of generator is",x.shape,"\n")
      return x

    def get_optimizer(self, module):
        """

        :param nn.ModuleList modules: [shared_net, discriminator, classifier, generator, composite_loss]
        :return:
        """
        opt = OptimizerParams()
        return torch.optim.Adam(module.parameters(),lr=opt.lr, betas=(opt.beta1,0.999), weight_decay=opt.weight_decay)

    def init_weights(self):
        def init_weight(m):
          cln = m.__class__.__name__
          if cln.find('Conv')!=-1:
            nn.init.normal_(m.weight.data,0.0,0.2)
          elif cln.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data,1.0,0.02)
            nn.init.constant_(m.bias.data,0)
        self.conv.apply(init_weight)


class CompositeLoss(nn.Module):
    class Params:
        dummy = 0

    def __init__(self, device, params):
        """

        :param torch.device device:
        :param CompositeLoss.Params params:
        """
        super(CompositeLoss, self).__init__()

    def forward(self, dscr_loss, cls_loss):
        #pass
        x = dscr_loss+cls_loss

        return x

class SaveCriteria:
    def __init__(self, status_df):
        """

        :param pd.DataFrame status_df:
        """
        self._opt_status_df = status_df.copy()

    def decide(self, status_df):
        """
        decide when to save new checkpoint while training based on training and validation stats
        following metrics are available:
      |   dscr_real |   cls_real |   cmp_real |   dscr_fake |   cls_fake |   cmp_fake |   cmp |   dscr_gen |   cls_gen |
         cmp_gen |   total_acc |   real_acc |   fake_acc |   fid |   is |

         where the first 10 are losses: dscr --> discriminator, cls --> classifier, gen --> generator,
         cmp --> composite, real --> real images, fake --> fake images

         acc --> classification accuracy
         is --> inception_score

        :param pd.DataFrame status_df:
        """

        save_weights = 0
        criterion = ''

        """total train accuracy over real+fake images"""
        if status_df['total_acc']['valid'] > self._opt_status_df['total_acc']['valid']:
            self._opt_status_df['total_acc']['valid'] = status_df['total_acc']['valid']
            save_weights = 1
            criterion = 'valid_acc'

        if status_df['total_acc']['train'] > self._opt_status_df['total_acc']['train']:
            self._opt_status_df['total_acc']['train'] = status_df['total_acc']['train']
            save_weights = 1
            criterion = 'train_acc'

        """composite loss on real images"""
        if status_df['cmp_real']['valid'] > self._opt_status_df['cmp_real']['valid']:
            self._opt_status_df['cmp_real']['valid'] = status_df['cmp_real']['valid']
            save_weights = 1
            criterion = 'valid_loss'

        if status_df['cmp_real']['train'] > self._opt_status_df['cmp_real']['train']:
            self._opt_status_df['cmp_real']['train'] = status_df['cmp_real']['train']
            save_weights = 1
            criterion = 'train_loss'

        return save_weights, criterion

In [ ]:
# new A5 Main file
import os
import re
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

import torch
from torchvision import transforms, datasets, version

print('Using pytorch version: {}'.format(torch.__version__))
print('Using torchvision version: {}'.format(version.__version__))

import torch.nn as nn
from torch.utils.data.sampler import *
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision.models.inception import inception_v3
from torchvision.utils import make_grid

"""
hack to deal with vagaries of Colab and Google Drive in batch mode
"""

"""
import shutil

def rreplace(s, old, new='', occurrence=1):
    li = s.rsplit(old, occurrence)
    return new.join(li)


script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = script_dir.replace(os.sep, '/') + '/'

postfixed_files = [_file for _file in os.listdir(script_dir) if
                   os.path.splitext(_file)[0].endswith(' (1)')]
if postfixed_files:
    print('postfixed_files: {}'.format(postfixed_files))

    for _file in postfixed_files:
        _dst_file = rreplace(_file, ' (1)', '')
        _src_path = os.path.join(script_dir, _file)
        _dst_path = os.path.join(script_dir, _dst_file)

        print('{} --> {}'.format(_file, _dst_file))
        shutil.move(_src_path, _dst_path)
"""

#from A5_submission import SharedNet, Discriminator, Classifier, Generator, CompositeLoss
#from A5_submission import TrainParams, SaveCriteria
from A5_utils import REAL_LABEL, FAKE_LABEL, REAL_UNLABELED, FAKE_UNLABELED, print_stats
from A5_utils import get_num_params, PartiallyLabeled, compute_fid, compute_inception_score, GANLosses, InceptionV3


class A5_Params:
    """


    :ivar gen_latent_size: Size of latent vector used as generator input

    :ivar fid_dims: which feature map to use for FID

    :ivar total_split: what fraction of training data to use; working on small subsets can be useful for debugging

    :ivar train_gen_metrics: toggle computing generator metrics FID and IS during training;
    disabling can speed up validation

    :ivar eval_gen_metrics: toggle computing generator metrics FID and IS during validation;
    disabling can speed up validation
    """

    def __init__(self):
        self.use_cuda = 1

        self.train_split = 0.76
        self.labeled_split = 0.2

        self.gen_latent_size = 100
        self.fid_dims = 2048

        self.train = TrainParams()

        """useful for debugging"""
        self.total_split = 1
        self.train_gen_metrics = 0
        self.eval_gen_metrics = 0

        """
        Module specific parameters
        """
        self.shared = SharedNet.Params()
        self.dscr = Discriminator.Params()
        self.cls = Classifier.Params()
        self.gen = Generator.Params()
        self.loss = CompositeLoss.Params()


def evaluate(all_modules, dataloader, composite_loss, inception_model, fid_model, upsample,
             device, params, n_classes=10):
    """

    :param nn.ModuleList all_modules:
    :param dataloader:
    :param CompositeLoss composite_loss:
    :param int vis:
    :param torch.device  device:
    :param A5_Params  params:
    :return:
    """
    all_modules.eval()

    shared_net, discriminator, \
    classifier, generator = all_modules  # type: SharedNet, Discriminator, Classifier, Generator

    n_batches = 0

    real_total = 0
    real_correct = 0

    fake_total = 0
    fake_correct = 0

    total_fid = 0
    total_inception_score = 0

    total_time = 0

    loss = GANLosses()
    loss_item = GANLosses()
    total_loss = GANLosses()
    mean_loss = GANLosses()

    fake_images = fake_images_grid = None
    real_images = real_images_grid = None

    print('evaluating...')

    n_batches = len(dataloader)

    with torch.no_grad():
        for batch_idx, (real_images, real_labels) in tqdm(enumerate(dataloader), total=n_batches):

            real_images = real_images.to(device)

            real_labels = real_labels.to(device)  # 0 to 9
            fake_labels = real_labels + n_classes  # 10 to 19

            """update discriminator  / classifier with real images"""
            dscr_real_labels = torch.full((real_labels.size(0),), REAL_LABEL, dtype=torch.float, device=device)
            dscr_fake_labels = torch.full((real_labels.size(0),), FAKE_LABEL, dtype=torch.float, device=device)

            noise = torch.randn(real_labels.size(0), params.gen_latent_size, 1, 1, device=device)

            start_t = time.time()

            shared_out_real = shared_net(real_images)
            dscr_out_real = discriminator(shared_out_real)
            cls_out_real = classifier(shared_out_real)

            """update discriminator  / classifier with fake images"""
            """Generate fake image batch with generator"""
            fake_images = generator(noise)

            shared_out_fake = shared_net(fake_images)
            dscr_out_fake = discriminator(shared_out_fake)
            cls_out_fake = classifier(shared_out_fake)

            """time only forward passes through nets"""
            end_t = time.time()

            test_time = end_t - start_t
            total_time += test_time

            """compute losses"""
            loss.dscr_real = discriminator.get_loss(dscr_out_real, dscr_real_labels)
            loss.cls_real = classifier.get_loss(cls_out_real, real_labels)
            loss.cmp_real = composite_loss(loss.dscr_real, loss.cls_real)

            loss.dscr_fake = discriminator.get_loss(dscr_out_fake, dscr_fake_labels)
            loss.cls_fake = classifier.get_loss(cls_out_fake, fake_labels)
            loss.cmp_fake = composite_loss(loss.dscr_fake, loss.cls_fake)

            """total loss for discriminator / classifier"""
            loss.cmp = loss.cmp_real + loss.cmp_fake

            """fake labels are real for generator loss"""
            loss.dscr_gen = discriminator.get_loss(dscr_out_fake, dscr_real_labels)
            loss.cls_gen = classifier.get_loss(cls_out_fake, real_labels)
            loss.cmp_gen = composite_loss(loss.dscr_gen, loss.cls_gen)

            """classification accuracy for real images"""
            _, pred_real = torch.max(cls_out_real.data, 1)
            pred_real = pred_real.squeeze()
            _correct = pred_real.eq(real_labels).sum().item()
            _total = real_labels.size(0)
            real_total += _total
            real_correct += _correct

            """classification accuracy for fake images"""
            _, pred_fake = torch.max(cls_out_fake.data, 1)
            pred_fake = pred_fake.squeeze()
            fake_total += fake_labels.size(0)
            fake_correct += pred_fake.eq(fake_labels).sum().item()

            """Generator metrics"""
            if params.eval_gen_metrics:
                """resize images to be compatible with inception"""
                real_images_up = upsample(real_images.detach())
                fake_images_up = upsample(fake_images.detach())

                inception_score = compute_inception_score(fake_images_up, inception_model)
                inception_score = inception_score[0]
                fid = compute_fid([real_images_up, fake_images_up], fid_model, device=device, dims=params.fid_dims)

                total_inception_score += inception_score
                total_fid += fid

            total_loss_dict = total_loss.__dict__
            for loss_type in loss.__dict__:
                loss_item.__dict__[loss_type] = loss.__dict__[loss_type]
                total_loss_dict[loss_type] += loss_item.__dict__[loss_type]

            n_batches += 1

    for loss_type in loss.__dict__:
        mean_loss.__dict__[loss_type] = total_loss.__dict__[loss_type] / n_batches

    if real_images is not None:
        real_images_grid = make_grid(real_images, padding=2, normalize=True)

    if fake_images is not None:
        fake_images_grid = make_grid(fake_images, padding=2, normalize=True)

    """mean classification accuracy for real, fake and real+fake"""
    real_acc = 100. * real_correct / real_total
    fake_acc = 100. * fake_correct / fake_total
    total_acc = 100. * (real_correct + fake_correct) / (real_total + fake_total)

    if params.eval_gen_metrics:
        total_inception_score /= n_batches
        total_fid /= n_batches
    else:
        total_inception_score = total_fid = -1

    return mean_loss, (real_acc, fake_acc, total_acc), \
           (total_inception_score, total_fid, real_images_grid, fake_images_grid), total_time


def main():
    """number of classes in FMNIST dataset"""
    n_classes = 10

    params = A5_Params()

    # optional command line argument parsing
    try:
        import paramparse
    except ImportError:
        pass
    else:
        paramparse.process(params)

    # init device
    if params.use_cuda and torch.cuda.is_available():
        device = torch.device("cuda")
        dtype = torch.cuda.FloatTensor
        print('Training on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        dtype = torch.FloatTensor
        print('Training on CPU')

    # load dataset
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_set = datasets.CIFAR10('data', train=True, download=True, transform=transform)

    test_set = datasets.CIFAR10('data', train=False, download=True, transform=transform)

    valid_set = datasets.CIFAR10('data', train=True, download=True, transform=transform)

    train_params = params.train

    num_train = int(len(train_set) * params.total_split)
    indices = list(range(num_train))
    split = int(np.floor(params.train_split * num_train))

    train_idx, valid_idx = indices[:split], indices[split:]
    train_set = PartiallyLabeled(train_set, train_idx, labeled_percent=params.labeled_split)

    print('Training samples: {}\n'
          'Validation samples: {}\n'
          'Labeled training samples: {}'
          ''.format(
        len(train_idx),
        len(valid_idx),
        train_set.n_labeled_data
    ))

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SequentialSampler(valid_idx)

    train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=train_params.batch_size, sampler=train_sampler,
                                                   num_workers=train_params.n_workers)
    valid_dataloader = torch.utils.data.DataLoader(valid_set, batch_size=24, sampler=valid_sampler,
                                                   num_workers=train_params.n_workers)
    test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=False,
                                                  num_workers=train_params.n_workers)

    # create all_modules
    shared_net = SharedNet(params.shared).to(device)  # type: SharedNet
    discriminator = Discriminator(params.dscr).to(device)  # type: Discriminator
    classifier = Classifier(params.cls).to(device)  # type: Classifier
    generator = Generator(params.gen, params.gen_latent_size).to(device)  # type: Generator
    composite_loss = CompositeLoss(device, params.loss).to(device)

    assert isinstance(shared_net, nn.Module), 'SharedNet must be an instance of nn.Module'
    assert isinstance(discriminator, nn.Module), 'Discriminator must be an instance of nn.Module'
    assert isinstance(classifier, nn.Module), 'Classifier must be an instance of nn.Module'
    assert isinstance(generator, nn.Module), 'Generator must be an instance of nn.Module'
    assert isinstance(composite_loss, nn.Module), 'CompositeLoss must be an instance of nn.Module'

    n_shared_params = get_num_params(shared_net)
    n_discriminator_params = get_num_params(discriminator)
    n_classifier_params = get_num_params(classifier)

    assert n_shared_params >= n_discriminator_params and n_shared_params >= n_classifier_params, \
        "Discriminator and classifier must have at least half their parameters shared"

    all_modules = nn.ModuleList((shared_net, discriminator, classifier, generator))  # type: nn.ModuleList

    # init weights
    shared_net.init_weights()
    discriminator.init_weights()
    generator.init_weights()
    classifier.init_weights()

    discriminator_opt = discriminator.get_optimizer(nn.ModuleList((shared_net, discriminator, classifier)))
    generator_opt = generator.get_optimizer(generator)

    weights_dir = os.path.dirname(train_params.weights_path)
    weights_name = os.path.basename(train_params.weights_path)

    if not os.path.isdir(weights_dir):
        os.makedirs(weights_dir)

    start_epoch = 0

    mean_loss = GANLosses()
    eval_metrics = list(mean_loss.__dict__.keys()) + ['total_acc', 'real_acc', 'fake_acc', 'fid', 'is']
    data_types = ['train', 'valid']

    status_df = pd.DataFrame(
        np.zeros((len(data_types), len(eval_metrics)), dtype=np.float32),
        index=data_types,
        columns=eval_metrics,
    )

    # load weights
    if train_params.load_weights:
        matching_ckpts = [k for k in os.listdir(weights_dir) if
                          os.path.isfile(os.path.join(weights_dir, k)) and
                          k.startswith(weights_name)]
        if not matching_ckpts:
            msg = 'No checkpoints found matching {} in {}'.format(weights_name, weights_dir)
            if train_params.load_weights == 1:
                raise IOError(msg)
            print(msg)
        else:
            matching_ckpts.sort(key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r'(\d+)', x)])

            weights_path = os.path.join(weights_dir, matching_ckpts[-1])

            chkpt = torch.load(weights_path, map_location=device)  # load checkpoint

            print('Loading weights from: {} with:\n'
                  '\tcriterion: {}\n'
                  '\tepoch: {}\n'
                  '\ttimestamp: {}\n'.format(
                weights_path,
                chkpt['criterion'],
                chkpt['epoch'],
                chkpt['timestamp']))

            status_df = chkpt['status_df']

            print('stats:')
            print_stats(status_df)
            print()

            shared_net.load_state_dict(chkpt['shared_net'])
            discriminator.load_state_dict(chkpt['discriminator'])
            generator.load_state_dict(chkpt['generator'])
            classifier.load_state_dict(chkpt['classifier'])
            generator_opt.load_state_dict(chkpt['generator_opt'])
            discriminator_opt.load_state_dict(chkpt['discriminator_opt'])

            start_epoch = chkpt['epoch'] + 1
    else:
        print('Training from scratch')

    if params.train_gen_metrics or params.eval_gen_metrics:
        inception_model = inception_v3(pretrained=True, transform_input=False, init_weights=False).type(dtype)
        """custom inception for FID"""
        block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[params.fid_dims]
        fid_model = InceptionV3(output_blocks=[block_idx], resize_input=False).to(device)
        """needed to resize images to be compatible with inception"""
        upsample = nn.Upsample(size=(299, 299), mode='bilinear', align_corners=True).type(dtype)
    else:
        inception_model = fid_model = upsample = None

    if train_params.load_weights != 1:
        """
        continue training
        """

        writer = SummaryWriter(log_dir=params.train.tb_path)
        print(f'Saving tensorboard summary to: {params.train.tb_path}')

        """
        training steps from: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
        """
        iter_id = 0

        """decide when to save weights"""
        save_criteria = SaveCriteria(status_df)

        if not params.train_gen_metrics:
            print('Generator metrics computation is disabled during training')

        if not params.eval_gen_metrics:
            print('Generator metrics computation is disabled during evaluation')

        for epoch in range(start_epoch, train_params.n_epochs):
            all_modules.train()

            real_total = 0
            real_correct = 0

            fake_total = 0
            fake_correct = 0

            batch_idx = 0

            train_fid = 0
            train_inception_score = 0

            total_loss = GANLosses()
            loss = GANLosses()
            loss_item = GANLosses()

            n_batches = len(train_dataloader)

            for batch_idx, (real_images, real_labels, is_labeled) in tqdm(enumerate(train_dataloader), total=n_batches):
                real_images = real_images.to(device)

                real_labels = real_labels.to(device)  # 0 to 9
                fake_labels = real_labels + n_classes  # 10 to 19

                is_labeled = is_labeled.squeeze()
                n_labeled = np.count_nonzero(is_labeled.detach().numpy())

                """remove labels for unlabeled images"""
                is_not_labeled = np.logical_not(is_labeled.cpu().numpy()).squeeze()
                real_labels[is_not_labeled] = REAL_UNLABELED
                fake_labels[is_not_labeled] = FAKE_UNLABELED

                """""""""""""""""""""""""""""""""""""""""""""""""""""
                train discriminator / classifier
                """""""""""""""""""""""""""""""""""""""""""""""""""""
                shared_net.zero_grad()
                discriminator.zero_grad()
                classifier.zero_grad()

                """update discriminator  / classifier with real images"""
                dscr_real_labels = torch.full((real_labels.size(0),), REAL_LABEL, dtype=torch.float, device=device)

                shared_out_real = shared_net(real_images)
                dscr_out_real = discriminator(shared_out_real)
                cls_out_real = classifier(shared_out_real)

                loss.dscr_real = discriminator.get_loss(dscr_out_real, dscr_real_labels)
                loss.cls_real = classifier.get_loss(cls_out_real, real_labels, is_labeled, n_labeled)
                loss.cmp_real = composite_loss(loss.dscr_real, loss.cls_real)

                """compute gradients for the real batch"""
                loss.cmp_real.backward()

                """update discriminator  / classifier with fake images"""
                dscr_fake_labels = torch.full((real_labels.size(0),), FAKE_LABEL, dtype=torch.float, device=device)

                noise = torch.randn(real_labels.size(0), params.gen_latent_size, 1, 1, device=device)

                """Generate fake image batch with generator"""
                fake_images = generator(noise)

                """we do not want generator gradients to be computed"""
                fake_images_no_grad = fake_images.detach()

                shared_out_fake = shared_net(fake_images_no_grad)
                dscr_out_fake = discriminator(shared_out_fake)
                cls_out_fake = classifier(shared_out_fake)

                loss.dscr_fake = discriminator.get_loss(dscr_out_fake, dscr_fake_labels)
                loss.cls_fake = classifier.get_loss(cls_out_fake, fake_labels, is_labeled, n_labeled)
                loss.cmp_fake = composite_loss(loss.dscr_fake, loss.cls_fake)

                """Add the gradients from the fake batch to the real one"""
                loss.cmp_fake.backward()

                """total loss for discriminator / classifier"""
                loss.cmp = loss.cmp_real + loss.cmp_fake

                """update discriminator and classifier parameters"""
                discriminator_opt.step()

                """""""""""""""""""""""""""""""""""""""""""""""""""""
                train generator
                """""""""""""""""""""""""""""""""""""""""""""""""""""
                generator.zero_grad()

                """Since we just updated the discriminator, perform another forward pass of all-fake batch through it"""
                shared_out_gen = shared_net(fake_images)
                dscr_out_gen = discriminator(shared_out_gen)
                cls_out_gen = classifier(shared_out_gen)

                """fake labels are real for generator loss"""
                loss.dscr_gen = discriminator.get_loss(dscr_out_gen, dscr_real_labels)
                loss.cls_gen = classifier.get_loss(cls_out_gen, real_labels, is_labeled, n_labeled)
                loss.cmp_gen = composite_loss(loss.dscr_gen, loss.cls_gen)

                loss.cmp_gen.backward()

                generator_opt.step()

                """""""""""""""""""""""""""""""""""""""""""""""""""""
                collect train statistics and add to tensorboard log
                """""""""""""""""""""""""""""""""""""""""""""""""""""
                total_loss_dict = total_loss.__dict__
                for loss_type in loss.__dict__:
                    loss_item.__dict__[loss_type] = loss.__dict__[loss_type]
                    total_loss_dict[loss_type] += loss_item.__dict__[loss_type]
                    writer.add_scalar(f'train_iter_loss/{loss_type}', loss_item.__dict__[loss_type], iter_id)

                """classification accuracy for real images"""
                _, pred_real = torch.max(cls_out_real.data, 1)
                pred_real = pred_real.squeeze()
                real_total += real_labels.size(0)
                real_correct += pred_real.eq(real_labels).sum().item()

                """classification accuracy for fake images"""
                _, pred_fake = torch.max(cls_out_fake.data, 1)
                pred_fake = pred_fake.squeeze()
                fake_total += fake_labels.size(0)
                fake_correct += pred_fake.eq(fake_labels).sum().item()

                if params.train_gen_metrics:
                    """resize images to be compatible with inception"""
                    real_images_up = upsample(real_images.detach())
                    fake_images_up = upsample(fake_images.detach())

                    inception_score = compute_inception_score(fake_images_up, inception_model)
                    """need only the mean KL Divergence"""
                    inception_score = inception_score[0]
                    fid = compute_fid([real_images_up, fake_images_up], fid_model, device=device, dims=params.fid_dims)

                    train_inception_score += inception_score
                    train_fid += fid

                    writer.add_scalar('train_iter/inception_score', inception_score, iter_id)
                    writer.add_scalar('train_iter/fid', fid, iter_id)

                real_images_grid = make_grid(real_images, padding=2, normalize=True)
                fake_images_grid = make_grid(fake_images, padding=2, normalize=True)

                writer.add_image('train/real_images', real_images_grid, epoch)
                writer.add_image('train/fake_images', fake_images_grid, epoch)

                iter_id += 1

            for loss_type in loss.__dict__:
                mean_loss.__dict__[loss_type] = total_loss.__dict__[loss_type] / (batch_idx + 1)
                writer.add_scalar(f'train_loss/{loss_type}', mean_loss.__dict__[loss_type], epoch)
                status_df[loss_type]['train'] = mean_loss.__dict__[loss_type]

            """mean classification accuracy for real, fake and real+fake"""
            real_acc = 100. * real_correct / real_total
            fake_acc = 100. * fake_correct / fake_total
            total_acc = 100. * (real_correct + fake_correct) / (real_total + fake_total)

            if params.train_gen_metrics:
                train_inception_score /= (batch_idx + 1)
                train_fid /= (batch_idx + 1)
                writer.add_scalar('train/fid', train_fid, epoch)
                writer.add_scalar('train/inception_score', train_inception_score, epoch)
                status_df['is']['train'] = train_inception_score
                status_df['fid']['train'] = train_fid

            writer.add_scalar('train/total_acc', total_acc, epoch)
            writer.add_scalar('train/real_acc', real_acc, epoch)
            writer.add_scalar('train/fake_acc', fake_acc, epoch)

            status_df['total_acc']['train'] = total_acc
            status_df['real_acc']['train'] = real_acc
            status_df['fake_acc']['train'] = fake_acc

            if epoch % train_params.validate_gap == 0:
                valid_loss, valid_acc, valid_gen, valid_time = evaluate(
                    all_modules, valid_dataloader, composite_loss, inception_model, fid_model, upsample, device, params)
                print('\nvalidation time: {:.3f}'.format(valid_time))

                valid_real_acc, valid_fake_acc, valid_total_acc = valid_acc
                valid_inception_score, valid_fid, valid_real_images_grid, valid_fake_images_grid, = valid_gen

                for loss_type in loss.__dict__:
                    writer.add_scalar(f'valid_loss/{loss_type}', valid_loss.__dict__[loss_type], epoch)
                    status_df[loss_type]['valid'] = valid_loss.__dict__[loss_type]

                writer.add_image('valid/real_images', valid_real_images_grid, epoch)
                writer.add_image('valid/fake_images', valid_fake_images_grid, epoch)

                writer.add_scalar('valid/total_acc', valid_total_acc, epoch)
                writer.add_scalar('valid/real_acc', valid_real_acc, epoch)
                writer.add_scalar('valid/fake_acc', valid_fake_acc, epoch)

                if params.eval_gen_metrics:
                    writer.add_scalar('valid/fid', valid_fid, epoch)
                    writer.add_scalar('valid/inception_score', valid_inception_score, epoch)

                    status_df['is']['valid'] = valid_inception_score
                    status_df['fid']['valid'] = valid_fid

                status_df['total_acc']['valid'] = valid_total_acc
                status_df['real_acc']['valid'] = valid_real_acc
                status_df['fake_acc']['valid'] = valid_fake_acc
            else:
                status_df.loc['valid', :] = 0

            save_weights, criterion = save_criteria.decide(status_df)

            print("Training results for epoch: {}:\n".format(epoch))
            print_stats(status_df)
            print()

            # Save checkpoint.
            if save_weights:
                model_dict = {
                    'shared_net': shared_net.state_dict(),
                    'discriminator': discriminator.state_dict(),
                    'generator': generator.state_dict(),
                    'classifier': classifier.state_dict(),
                    'discriminator_opt': discriminator_opt.state_dict(),
                    'generator_opt': generator_opt.state_dict(),
                    'status_df': status_df,
                    'criterion': criterion,
                    'epoch': epoch,
                    'timestamp': datetime.now().strftime("%y/%m/%d %H:%M:%S"),
                }
                weights_path = '{}.{:d}'.format(train_params.weights_path, epoch)

                print(f'Saving weights for criterion {criterion} to {weights_path}')
                torch.save(model_dict, weights_path)

    print('Testing...')
    test_loss, test_acc, test_gen, test_time = evaluate(
        all_modules, test_dataloader, composite_loss, inception_model, fid_model, upsample, device, params)

    test_real_acc, test_fake_acc, test_total_acc = test_acc
    test_inception_score, test_fid, test_real_images_grid, test_fake_images_grid = test_gen

    test_df = pd.DataFrame(
        np.zeros((1, len(eval_metrics)), dtype=np.float32),
        index=('test',),
        columns=eval_metrics,
    )
    for loss_type in test_loss.__dict__:
        test_df[loss_type]['test'] = test_loss.__dict__[loss_type]

    test_df['total_acc']['test'] = test_total_acc
    test_df['real_acc']['test'] = test_real_acc
    test_df['fake_acc']['test'] = test_fake_acc
    test_df['is']['test'] = test_inception_score
    test_df['fid']['test'] = test_fid

    print("Test Results:\n")
    print_stats(test_df)
    print()
    print('Test Time: {:.3f} sec'.format(test_time))


if __name__ == '__main__':
    main()

Using pytorch version: 1.7.0+cu101
Using torchvision version: 0.8.1+cu101
Training on GPU: Tesla T4


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
Files already downloaded and verified
Training samples: 38000
Validation samples: 12000
Labeled training samples: 7600
Training from scratch


  0%|          | 0/297 [00:00<?, ?it/s]

Saving tensorboard summary to: ./tensorboard
Generator metrics computation is disabled during training
Generator metrics computation is disabled during evaluation


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  2%|▏         | 10/500 [00:00<00:04, 98.38it/s]

...

Training results for epoch: 48:

|       |   dscr_real |   cls_real |   cmp_real |   dscr_fake |   cls_fake |   cmp_fake |   cmp |   dscr_gen |   cls_gen |   cmp_gen |   total_acc |   real_acc |   fake_acc |   fid |    is |
|-------+-------------+------------+------------+-------------+------------+------------+-------+------------+-----------+-----------+-------------+------------+------------+-------+-------|
| train |       0.000 |      2.145 |      2.145 |       0.000 |      2.949 |      2.949 | 5.094 |     10.247 |     3.048 |    13.295 |      10.350 |     18.703 |      1.997 | 0.000 | 0.000 |
| valid |       0.000 |      0.000 |      0.000 |       0.000 |      0.000 |      0.000 | 0.000 |      0.000 |     0.000 |     0.000 |       0.000 |      0.000 |      0.000 | 0.000 | 0.000 |



  2%|▏         | 18/1000 [00:00<00:05, 174.82it/s]

Training results for epoch: 49:

|       |   dscr_real |   cls_real |   cmp_real |   dscr_fake |   cls_fake |   cmp_fake |   cmp |   dscr_gen |   cls_gen |   cmp_gen |   total_acc |   real_acc |   fake_acc |   fid |    is |
|-------+-------------+------------+------------+-------------+------------+------------+-------+------------+-----------+-----------+-------------+------------+------------+-------+-------|
| train |       0.000 |      2.199 |      2.199 |       0.000 |      2.950 |      2.950 | 5.149 |      9.767 |     3.049 |    12.816 |      10.013 |     17.968 |      2.058 | 0.000 | 0.000 |
| valid |       0.000 |      0.000 |      0.000 |       0.000 |      0.000 |      0.000 | 0.000 |      0.000 |     0.000 |     0.000 |       0.000 |      0.000 |      0.000 | 0.000 | 0.000 |

Testing...
evaluating...


100%|██████████| 1000/1000 [00:06<00:00, 151.90it/s]

Test Results:

|      |   dscr_real |   cls_real |   cmp_real |   dscr_fake |   cls_fake |   cmp_fake |   cmp |   dscr_gen |   cls_gen |   cmp_gen |   total_acc |   real_acc |   fake_acc |    fid |     is |
|------+-------------+------------+------------+-------------+------------+------------+-------+------------+-----------+-----------+-------------+------------+------------+--------+--------|
| test |       0.000 |      1.479 |      1.479 |       0.002 |      1.478 |      1.480 | 2.959 |      2.792 |     1.527 |     4.320 |      10.570 |     11.140 |     10.000 | -1.000 | -1.000 |

Test Time: 2.236 sec


loss 直接加的话test结果real-acc = 32.xx


loss 取均值的结果 = ?

In [ ]:
#@title
import os
import re
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

import torch
from torchvision import transforms, datasets, version

print('Using pytorch version: {}'.format(torch.__version__))
print('Using torchvision version: {}'.format(version.__version__))

import torch.nn as nn
from torch.utils.data.sampler import *
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision.models.inception import inception_v3
from torchvision.utils import make_grid

"""
hack to deal with vagaries of Colab and Google Drive in batch mode
"""

"""
import shutil

def rreplace(s, old, new='', occurrence=1):
    li = s.rsplit(old, occurrence)
    return new.join(li)


script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = script_dir.replace(os.sep, '/') + '/'

postfixed_files = [_file for _file in os.listdir(script_dir) if
                   os.path.splitext(_file)[0].endswith(' (1)')]
if postfixed_files:
    print('postfixed_files: {}'.format(postfixed_files))

    for _file in postfixed_files:
        _dst_file = rreplace(_file, ' (1)', '')
        _src_path = os.path.join(script_dir, _file)
        _dst_path = os.path.join(script_dir, _dst_file)

        print('{} --> {}'.format(_file, _dst_file))
        shutil.move(_src_path, _dst_path)
"""

#from A5_submission import SharedNet, Discriminator, Classifier, Generator, CompositeLoss
#from A5_submission import TrainParams, SaveCriteria
from A5_utils import REAL_LABEL, FAKE_LABEL, REAL_UNLABELED, FAKE_UNLABELED, print_stats
from A5_utils import get_num_params, PartiallyLabeled, compute_fid, compute_inception_score, GANLosses, InceptionV3


class A5_Params:
    """


    :ivar gen_latent_size: Size of latent vector used as generator input

    :ivar fid_dims: which feature map to use for FID

    :ivar total_split: what fraction of training data to use; working on small subsets can be useful for debugging

    :ivar train_gen_metrics: toggle computing generator metrics FID and IS during training;
    disabling can speed up validation

    :ivar eval_gen_metrics: toggle computing generator metrics FID and IS during validation;
    disabling can speed up validation
    """

    def __init__(self):
        self.use_cuda = 1

        self.train_split = 0.76
        self.labeled_split = 0.2

        self.gen_latent_size = 100
        self.fid_dims = 2048

        self.train = TrainParams()

        """useful for debugging"""
        self.total_split = 0.1
        self.train_gen_metrics = 0
        self.eval_gen_metrics = 0

        """
        Module specific parameters
        """
        self.shared = SharedNet.Params()
        self.dscr = Discriminator.Params()
        self.cls = Classifier.Params()
        self.gen = Generator.Params()
        self.loss = CompositeLoss.Params()


def evaluate(all_modules, dataloader, composite_loss, inception_model, fid_model, upsample,
             device, params, n_classes=10):
    """

    :param nn.ModuleList all_modules:
    :param dataloader:
    :param CompositeLoss composite_loss:
    :param int vis:
    :param torch.device  device:
    :param A5_Params  params:
    :return:
    """
    all_modules.eval()

    shared_net, discriminator, \
    classifier, generator = all_modules  # type: SharedNet, Discriminator, Classifier, Generator

    n_batches = 0

    real_total = 0
    real_correct = 0

    fake_total = 0
    fake_correct = 0

    total_fid = 0
    total_inception_score = 0

    total_time = 0

    loss = GANLosses()
    loss_item = GANLosses()
    total_loss = GANLosses()
    mean_loss = GANLosses()

    fake_images = fake_images_grid = None
    real_images = real_images_grid = None

    print('evaluating...')

    n_batches = len(dataloader)

    with torch.no_grad():
        for batch_idx, (real_images, real_labels) in tqdm(enumerate(dataloader), total=n_batches):

            real_images = real_images.to(device)

            real_labels = real_labels.to(device)  # 0 to 9
            fake_labels = real_labels + n_classes  # 10 to 19

            """update discriminator  / classifier with real images"""
            dscr_real_labels = torch.full((real_labels.size(0),), REAL_LABEL, dtype=torch.float, device=device)
            dscr_fake_labels = torch.full((real_labels.size(0),), FAKE_LABEL, dtype=torch.float, device=device)

            noise = torch.randn(real_labels.size(0), params.gen_latent_size, 1, 1, device=device)

            start_t = time.time()

            shared_out_real = shared_net(real_images)
            dscr_out_real = discriminator(shared_out_real)
            cls_out_real = classifier(shared_out_real)

            """update discriminator  / classifier with fake images"""
            """Generate fake image batch with generator"""
            fake_images = generator(noise)

            shared_out_fake = shared_net(fake_images)
            dscr_out_fake = discriminator(shared_out_fake)
            cls_out_fake = classifier(shared_out_fake)

            """time only forward passes through nets"""
            end_t = time.time()

            test_time = end_t - start_t
            total_time += test_time

            """compute losses"""
            loss.dscr_real = discriminator.get_loss(dscr_out_real, dscr_real_labels)
            loss.cls_real = classifier.get_loss(cls_out_real, real_labels)
            loss.cmp_real = composite_loss(loss.dscr_real, loss.cls_real)

            loss.dscr_fake = discriminator.get_loss(dscr_out_fake, dscr_fake_labels)
            loss.cls_fake = classifier.get_loss(cls_out_fake, fake_labels)
            loss.cmp_fake = composite_loss(loss.dscr_fake, loss.cls_fake)

            """total loss for discriminator / classifier"""
            loss.cmp = loss.cmp_real + loss.cmp_fake

            """fake labels are real for generator loss"""
            loss.dscr_gen = discriminator.get_loss(dscr_out_fake, dscr_real_labels)
            loss.cls_gen = classifier.get_loss(cls_out_fake, real_labels)
            loss.cmp_gen = composite_loss(loss.dscr_gen, loss.cls_gen)

            """classification accuracy for real images"""
            _, pred_real = torch.max(cls_out_real.data, 1)
            pred_real = pred_real.squeeze()
            _correct = pred_real.eq(real_labels).sum().item()
            _total = real_labels.size(0)
            real_total += _total
            real_correct += _correct

            """classification accuracy for fake images"""
            _, pred_fake = torch.max(cls_out_fake.data, 1)
            pred_fake = pred_fake.squeeze()
            fake_total += fake_labels.size(0)
            fake_correct += pred_fake.eq(fake_labels).sum().item()

            """Generator metrics"""
            if params.eval_gen_metrics:
                """resize images to be compatible with inception"""
                real_images_up = upsample(real_images.detach())
                fake_images_up = upsample(fake_images.detach())

                inception_score = compute_inception_score(fake_images_up, inception_model)
                inception_score = inception_score[0]
                fid = compute_fid([real_images_up, fake_images_up], fid_model, device=device, dims=params.fid_dims)

                total_inception_score += inception_score
                total_fid += fid

            total_loss_dict = total_loss.__dict__
            for loss_type in loss.__dict__:
                loss_item.__dict__[loss_type] = loss.__dict__[loss_type]
                total_loss_dict[loss_type] += loss_item.__dict__[loss_type]

            n_batches += 1

    for loss_type in loss.__dict__:
        mean_loss.__dict__[loss_type] = total_loss.__dict__[loss_type] / n_batches

    if real_images is not None:
        real_images_grid = make_grid(real_images, padding=2, normalize=True)

    if fake_images is not None:
        fake_images_grid = make_grid(fake_images, padding=2, normalize=True)

    """mean classification accuracy for real, fake and real+fake"""
    real_acc = 100. * real_correct / real_total
    fake_acc = 100. * fake_correct / fake_total
    total_acc = 100. * (real_correct + fake_correct) / (real_total + fake_total)

    if params.eval_gen_metrics:
        total_inception_score /= n_batches
        total_fid /= n_batches
    else:
        total_inception_score = total_fid = -1

    return mean_loss, (real_acc, fake_acc, total_acc), \
           (total_inception_score, total_fid, real_images_grid, fake_images_grid), total_time


def main():
    """number of classes in FMNIST dataset"""
    n_classes = 10

    params = A5_Params()

    # optional command line argument parsing
    try:
        import paramparse
    except ImportError:
        pass
    else:
        paramparse.process(params)

    # init device
    if params.use_cuda and torch.cuda.is_available():
        device = torch.device("cuda")
        dtype = torch.cuda.FloatTensor
        print('Training on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        dtype = torch.FloatTensor
        print('Training on CPU')

    # load dataset
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_set = datasets.CIFAR10('data', train=True, download=True, transform=transform)

    test_set = datasets.CIFAR10('data', train=False, download=True, transform=transform)

    valid_set = datasets.CIFAR10('data', train=True, download=True, transform=transform)

    train_params = params.train

    num_train = int(len(train_set) * params.total_split)
    indices = list(range(num_train))
    split = int(np.floor(params.train_split * num_train))

    train_idx, valid_idx = indices[:split], indices[split:]
    train_set = PartiallyLabeled(train_set, train_idx, labeled_percent=params.labeled_split)

    print('Training samples: {}\n'
          'Validation samples: {}\n'
          'Labeled training samples: {}'
          ''.format(
        len(train_idx),
        len(valid_idx),
        train_set.n_labeled_data
    ))

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SequentialSampler(valid_idx)

    train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=train_params.batch_size, sampler=train_sampler,
                                                   num_workers=train_params.n_workers)
    valid_dataloader = torch.utils.data.DataLoader(valid_set, batch_size=24, sampler=valid_sampler,
                                                   num_workers=train_params.n_workers)
    test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=False,
                                                  num_workers=train_params.n_workers)

    # create all_modules
    shared_net = SharedNet(params.shared).to(device)  # type: SharedNet
    discriminator = Discriminator(params.dscr).to(device)  # type: Discriminator
    classifier = Classifier(params.cls).to(device)  # type: Classifier
    generator = Generator(params.gen, params.gen_latent_size).to(device)  # type: Generator
    composite_loss = CompositeLoss(device, params.loss).to(device)

    assert isinstance(shared_net, nn.Module), 'SharedNet must be an instance of nn.Module'
    assert isinstance(discriminator, nn.Module), 'Discriminator must be an instance of nn.Module'
    assert isinstance(classifier, nn.Module), 'Classifier must be an instance of nn.Module'
    assert isinstance(generator, nn.Module), 'Generator must be an instance of nn.Module'
    assert isinstance(composite_loss, nn.Module), 'CompositeLoss must be an instance of nn.Module'

    n_shared_params = get_num_params(shared_net)
    n_discriminator_params = get_num_params(discriminator)
    n_classifier_params = get_num_params(classifier)

    assert n_shared_params >= n_discriminator_params and n_shared_params >= n_classifier_params, \
        "Discriminator and classifier must have at least half their parameters shared"

    all_modules = nn.ModuleList((shared_net, discriminator, classifier, generator))  # type: nn.ModuleList

    # init weights
    shared_net.init_weights()
    discriminator.init_weights()
    generator.init_weights()
    classifier.init_weights()

    discriminator_opt = discriminator.get_optimizer(nn.ModuleList((shared_net, discriminator, classifier)))
    generator_opt = generator.get_optimizer(generator)

    weights_dir = os.path.dirname(train_params.weights_path)
    weights_name = os.path.basename(train_params.weights_path)

    if not os.path.isdir(weights_dir):
        os.makedirs(weights_dir)

    start_epoch = 0

    mean_loss = GANLosses()
    eval_metrics = list(mean_loss.__dict__.keys()) + ['total_acc', 'real_acc', 'fake_acc', 'fid', 'is']
    data_types = ['train', 'valid']

    status_df = pd.DataFrame(
        np.zeros((len(data_types), len(eval_metrics)), dtype=np.float32),
        index=data_types,
        columns=eval_metrics,
    )

    # load weights
    if train_params.load_weights:
        matching_ckpts = [k for k in os.listdir(weights_dir) if
                          os.path.isfile(os.path.join(weights_dir, k)) and
                          k.startswith(weights_name)]
        if not matching_ckpts:
            msg = 'No checkpoints found matching {} in {}'.format(weights_name, weights_dir)
            if train_params.load_weights == 1:
                raise IOError(msg)
            print(msg)
        else:
            matching_ckpts.sort(key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r'(\d+)', x)])

            weights_path = os.path.join(weights_dir, matching_ckpts[-1])

            chkpt = torch.load(weights_path, map_location=device)  # load checkpoint

            print('Loading weights from: {} with:\n'
                  '\tcriterion: {}\n'
                  '\tepoch: {}\n'
                  '\ttimestamp: {}\n'.format(
                weights_path,
                chkpt['criterion'],
                chkpt['epoch'],
                chkpt['timestamp']))

            status_df = chkpt['status_df']

            print('stats:')
            print_stats(status_df)
            print()

            discriminator.load_state_dict(chkpt['discriminator'])
            generator.load_state_dict(chkpt['generator'])
            classifier.load_state_dict(chkpt['classifier'])
            generator_opt.load_state_dict(chkpt['generator_opt'])
            discriminator_opt.load_state_dict(chkpt['discriminator_opt'])

            start_epoch = chkpt['epoch'] + 1
    else:
        print('Training from scratch')

    if params.train_gen_metrics or params.eval_gen_metrics:
        inception_model = inception_v3(pretrained=True, transform_input=False, init_weights=False).type(dtype)
        """custom inception for FID"""
        block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[params.fid_dims]
        fid_model = InceptionV3(output_blocks=[block_idx], resize_input=False).to(device)
        """needed to resize images to be compatible with inception"""
        upsample = nn.Upsample(size=(299, 299), mode='bilinear', align_corners=True).type(dtype)
    else:
        inception_model = fid_model = upsample = None

    if train_params.load_weights != 1:
        """
        continue training
        """

        writer = SummaryWriter(log_dir=params.train.tb_path)
        print(f'Saving tensorboard summary to: {params.train.tb_path}')

        """
        training steps from: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
        """
        iter_id = 0

        """decide when to save weights"""
        save_criteria = SaveCriteria(status_df)

        if not params.train_gen_metrics:
            print('Generator metrics computation is disabled during training')

        if not params.eval_gen_metrics:
            print('Generator metrics computation is disabled during evaluation')

        for epoch in range(start_epoch, train_params.n_epochs):
            all_modules.train()

            real_total = 0
            real_correct = 0

            fake_total = 0
            fake_correct = 0

            batch_idx = 0

            train_fid = 0
            train_inception_score = 0

            total_loss = GANLosses()
            loss = GANLosses()
            loss_item = GANLosses()

            n_batches = len(train_dataloader)

            for batch_idx, (real_images, real_labels, is_labeled) in tqdm(enumerate(train_dataloader), total=n_batches):
                real_images = real_images.to(device)

                real_labels = real_labels.to(device)  # 0 to 9
                fake_labels = real_labels + n_classes  # 10 to 19

                is_labeled = is_labeled.squeeze()
                n_labeled = np.count_nonzero(is_labeled.detach().numpy())

                """remove labels for unlabeled images"""
                is_not_labeled = np.logical_not(is_labeled.cpu().numpy()).squeeze()
                real_labels[is_not_labeled] = REAL_UNLABELED
                fake_labels[is_not_labeled] = FAKE_UNLABELED

                """""""""""""""""""""""""""""""""""""""""""""""""""""
                train discriminator / classifier
                """""""""""""""""""""""""""""""""""""""""""""""""""""
                shared_net.zero_grad()
                discriminator.zero_grad()
                classifier.zero_grad()

                """update discriminator  / classifier with real images"""
                dscr_real_labels = torch.full((real_labels.size(0),), REAL_LABEL, dtype=torch.float, device=device)

                shared_out_real = shared_net(real_images)
                dscr_out_real = discriminator(shared_out_real)
                cls_out_real = classifier(shared_out_real)

                loss.dscr_real = discriminator.get_loss(dscr_out_real, dscr_real_labels)
                loss.cls_real = classifier.get_loss(cls_out_real, real_labels, is_labeled, n_labeled)
                x = composite_loss(loss.dscr_real, loss.cls_real)
                #print('the composite loss in main is',x)
                loss.cmp_real = composite_loss(loss.dscr_real, loss.cls_real)

                """compute gradients for the real batch"""
                loss.cmp_real.backward()

                """update discriminator  / classifier with fake images"""
                dscr_fake_labels = torch.full((real_labels.size(0),), FAKE_LABEL, dtype=torch.float, device=device)

                noise = torch.randn(real_labels.size(0), params.gen_latent_size, 1, 1, device=device)

                """Generate fake image batch with generator"""
                fake_images = generator(noise)

                """we do not want generator gradients to be computed"""
                fake_images_no_grad = fake_images.detach()

                shared_out_fake = shared_net(fake_images_no_grad)
                dscr_out_fake = discriminator(shared_out_fake)
                cls_out_fake = classifier(shared_out_fake)

                loss.dscr_fake = discriminator.get_loss(dscr_out_fake, dscr_fake_labels)
                loss.cls_fake = classifier.get_loss(cls_out_fake, fake_labels, is_labeled, n_labeled)
                loss.cmp_fake = composite_loss(loss.dscr_fake, loss.cls_fake)

                """Add the gradients from the fake batch to the real one"""
                loss.cmp_fake.backward()

                """total loss for discriminator / classifier"""
                loss.cmp = loss.cmp_real + loss.cmp_fake

                """update discriminator and classifier parameters"""
                discriminator_opt.step()

                """""""""""""""""""""""""""""""""""""""""""""""""""""
                train generator
                """""""""""""""""""""""""""""""""""""""""""""""""""""
                generator.zero_grad()

                """Since we just updated the discriminator, perform another forward pass of all-fake batch through it"""
                shared_out_gen = shared_net(fake_images)
                dscr_out_gen = discriminator(shared_out_gen)
                cls_out_gen = classifier(shared_out_gen)

                """fake labels are real for generator loss"""
                loss.dscr_gen = discriminator.get_loss(dscr_out_gen, dscr_real_labels)
                loss.cls_gen = classifier.get_loss(cls_out_gen, real_labels, is_labeled, n_labeled)
                loss.cmp_gen = composite_loss(loss.dscr_gen, loss.cls_gen)

                loss.cmp_gen.backward()

                generator_opt.step()

                """""""""""""""""""""""""""""""""""""""""""""""""""""
                collect train statistics and add to tensorboard log
                """""""""""""""""""""""""""""""""""""""""""""""""""""
                total_loss_dict = total_loss.__dict__
                for loss_type in loss.__dict__:
                    loss_item.__dict__[loss_type] = loss.__dict__[loss_type]
                    total_loss_dict[loss_type] += loss_item.__dict__[loss_type]
                    writer.add_scalar(f'train_iter_loss/{loss_type}', loss_item.__dict__[loss_type], iter_id)

                """classification accuracy for real images"""
                _, pred_real = torch.max(cls_out_real.data, 1)
                pred_real = pred_real.squeeze()
                real_total += real_labels.size(0)
                real_correct += pred_real.eq(real_labels).sum().item()

                """classification accuracy for fake images"""
                _, pred_fake = torch.max(cls_out_fake.data, 1)
                pred_fake = pred_fake.squeeze()
                fake_total += fake_labels.size(0)
                fake_correct += pred_fake.eq(fake_labels).sum().item()

                if params.train_gen_metrics:
                    """resize images to be compatible with inception"""
                    real_images_up = upsample(real_images.detach())
                    fake_images_up = upsample(fake_images.detach())

                    inception_score = compute_inception_score(fake_images_up, inception_model)
                    """need only the mean KL Divergence"""
                    inception_score = inception_score[0]
                    fid = compute_fid([real_images_up, fake_images_up], fid_model, device=device, dims=params.fid_dims)

                    train_inception_score += inception_score
                    train_fid += fid

                    writer.add_scalar('train_iter/inception_score', inception_score, iter_id)
                    writer.add_scalar('train_iter/fid', fid, iter_id)

                real_images_grid = make_grid(real_images, padding=2, normalize=True)
                fake_images_grid = make_grid(fake_images, padding=2, normalize=True)

                writer.add_image('train/real_images', real_images_grid, epoch)
                writer.add_image('train/fake_images', fake_images_grid, epoch)

                iter_id += 1

            for loss_type in loss.__dict__:
                mean_loss.__dict__[loss_type] = total_loss.__dict__[loss_type] / (batch_idx + 1)
                writer.add_scalar(f'train_loss/{loss_type}', mean_loss.__dict__[loss_type], epoch)
                status_df[loss_type]['train'] = mean_loss.__dict__[loss_type]

            """mean classification accuracy for real, fake and real+fake"""
            real_acc = 100. * real_correct / real_total
            fake_acc = 100. * fake_correct / fake_total
            total_acc = 100. * (real_correct + fake_correct) / (real_total + fake_total)

            if params.train_gen_metrics:
                train_inception_score /= (batch_idx + 1)
                train_fid /= (batch_idx + 1)
                writer.add_scalar('train/fid', train_fid, epoch)
                writer.add_scalar('train/inception_score', train_inception_score, epoch)
                status_df['is']['train'] = train_inception_score
                status_df['fid']['train'] = train_fid

            writer.add_scalar('train/total_acc', total_acc, epoch)
            writer.add_scalar('train/real_acc', real_acc, epoch)
            writer.add_scalar('train/fake_acc', fake_acc, epoch)

            status_df['total_acc']['train'] = total_acc
            status_df['real_acc']['train'] = real_acc
            status_df['fake_acc']['train'] = fake_acc

            if epoch % train_params.validate_gap == 0:
                valid_loss, valid_acc, valid_gen, valid_time = evaluate(
                    all_modules, valid_dataloader, composite_loss, inception_model, fid_model, upsample, device, params)
                print('\nvalidation time: {:.3f}'.format(valid_time))

                valid_real_acc, valid_fake_acc, valid_total_acc = valid_acc
                valid_inception_score, valid_fid, valid_real_images_grid, valid_fake_images_grid, = valid_gen

                for loss_type in loss.__dict__:
                    writer.add_scalar(f'valid_loss/{loss_type}', valid_loss.__dict__[loss_type], epoch)
                    status_df[loss_type]['valid'] = valid_loss.__dict__[loss_type]

                writer.add_image('valid/real_images', valid_real_images_grid, epoch)
                writer.add_image('valid/fake_images', valid_fake_images_grid, epoch)

                writer.add_scalar('valid/total_acc', valid_total_acc, epoch)
                writer.add_scalar('valid/real_acc', valid_real_acc, epoch)
                writer.add_scalar('valid/fake_acc', valid_fake_acc, epoch)

                if params.eval_gen_metrics:
                    writer.add_scalar('valid/fid', valid_fid, epoch)
                    writer.add_scalar('valid/inception_score', valid_inception_score, epoch)

                    status_df['is']['valid'] = valid_inception_score
                    status_df['fid']['valid'] = valid_fid

                status_df['total_acc']['valid'] = valid_total_acc
                status_df['real_acc']['valid'] = valid_real_acc
                status_df['fake_acc']['valid'] = valid_fake_acc
            else:
                status_df.loc['valid', :] = 0

            save_weights, criterion = save_criteria.decide(status_df)

            print("Training results for epoch: {}:\n".format(epoch))
            print_stats(status_df)
            print()

            # Save checkpoint.
            if save_weights:
                model_dict = {
                    'discriminator': discriminator.state_dict(),
                    'generator': generator.state_dict(),
                    'classifier': classifier.state_dict(),
                    'discriminator_opt': discriminator_opt.state_dict(),
                    'generator_opt': generator_opt.state_dict(),
                    'status_df': status_df,
                    'criterion': criterion,
                    'epoch': epoch,
                    'timestamp': datetime.now().strftime("%y/%m/%d %H:%M:%S"),
                }
                weights_path = '{}.{:d}'.format(train_params.weights_path, epoch)

                print(f'Saving weights for criterion {criterion} to {weights_path}')
                torch.save(model_dict, weights_path)

    print('Testing...')
    test_loss, test_acc, test_gen, test_time = evaluate(
        all_modules, test_dataloader, composite_loss, inception_model, fid_model, upsample, device, params)

    test_real_acc, test_fake_acc, test_total_acc = test_acc
    test_inception_score, test_fid, test_real_images_grid, test_fake_images_grid = test_gen

    test_df = pd.DataFrame(
        np.zeros((1, len(eval_metrics)), dtype=np.float32),
        index=('test',),
        columns=eval_metrics,
    )
    for loss_type in test_loss.__dict__:
        test_df[loss_type]['test'] = test_loss.__dict__[loss_type]

    test_df['total_acc']['test'] = test_total_acc
    test_df['real_acc']['test'] = test_real_acc
    test_df['fake_acc']['test'] = test_fake_acc
    test_df['is']['test'] = test_inception_score
    test_df['fid']['test'] = test_fid

    print("Test Results:\n")
    print_stats(test_df)
    print()
    print('Test Time: {:.3f} sec'.format(test_time))


if __name__ == '__main__':
    main()